# Kaggle Competition: House Prices 
## Author: Justine Schabel 

### Data Exploration

First we need to import the required libraries. 

In [28]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Libraries for reading, cleaning and plotting the dataa
import numpy as np 
import pandas as pd 
import csv
import matplotlib.pyplot as plt
import seaborn as sns

# Libraries for models 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.linear_model import LinearRegression

In [2]:
# Force numpy not to truncate output 
np.set_printoptions(threshold=100)

In [3]:
# Force pandas not to truncate output 
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

Read the data from the csv data files we downloaded from Kaggle

In [4]:
# Read in training data 
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

Split the training data into 80% training data and 20% dev data. We have 1168 training examples, 292 dev examples, 1495 test examples and 81 features (including the label).

In [5]:
# Split training data (labeled) into 80% training and 20% dev) and randomly sample 
training_data = train_df.sample(frac=0.8)
dev_data = train_df.drop(training_data.index)

# Examine shape of data sets
print("Training Set Size: ", training_data.shape)
print("Dev Set Size: ", dev_data.shape)
print("Test Set Size: ", test_df.shape)

Training Set Size:  (1168, 81)
Dev Set Size:  (292, 81)
Test Set Size:  (1459, 80)


We can also breifly look at a summary of the features. 
- The count attribute shows us that there are columns with missing data - we will need to manage NAs and look for outliers (ex. 9999 for NA)
- The mean attribute shows us that the average value of the features varies widely - we might consider standardizing 

In [6]:
# Briefly examine feature attributes for the training data 
training_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1168.000000,1168.000000,955.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1161.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1102.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000
mean,720.967466,57.551370,69.854450,10455.229452,6.128425,5.577911,1971.928938,1985.370719,102.824289,444.156678,49.229452,563.974315,1057.360445,1158.101884,342.011130,5.311644,1505.424658,0.428082,0.058219,1.566781,0.387842,2.856164,1.044521,6.477740,0.603596,1979.133394,1.772260,473.627568,94.059932,46.016267,21.696062,3.736301,14.987158,0.993151,48.410959,6.297089,2007.844178,180341.136130
std,425.354878,43.109105,23.450308,9826.869743,1.340406,1.097208,30.181188,20.360851,176.344349,433.039518,169.423106,440.111579,417.355147,378.425723,429.678949,44.491823,497.399012,0.515367,0.241463,0.546698,0.503039,0.791902,0.210449,1.563131,0.636955,24.374675,0.739057,209.740463,124.252934,65.808326,60.831986,30.944165,55.360939,24.154883,545.980770,2.704199,1.328496,75892.085404
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,372.000000,0.000000,0.000000,480.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,351.750000,20.000000,59.500000,7526.250000,5.000000,5.000000,1954.000000,1968.000000,0.000000,0.000000,0.000000,222.500000,794.000000,866.000000,0.000000,0.000000,1133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1962.000000,1.000000,338.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,130000.000000
50%,715.500000,50.000000,70.000000,9525.000000,6.000000,5.000000,1974.500000,1994.000000,0.000000,387.000000,0.000000,471.500000,991.000000,1080.000000,0.000000,0.000000,1457.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.500000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,165000.000000
75%,1090.500000,70.000000,80.000000,11515.500000,7.000000,6.000000,2001.000000,2004.000000,168.000000,719.250000,0.000000,795.250000,1302.000000,1382.250000,728.000000,0.000000,1767.250000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,65.250000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1459.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2009.000000,2010.000000,1600.000000,2260.000000,1474.000000,2336.000000,3206.000000,3228.000000,1872.000000,513.000000,4676.000000,2.000000,2.000000,3.000000,2.000000,6.000000,2.000000,12.000000,3.000000,2010.000000,4.000000,1390.000000,857.000000,547.000000,552.000000,508.000000,480.000000,648.000000,15500.000000,12.000000,2010.000000,755000.000000


I will also consolodate our findings from the data_description file. Most features are either alphabetic (categorical) or numeric.

- Categorical Features:
    - Numeric:
        - MSSubCass: Identifies the type of dwelling involved in the sale
        - OverallQual: Rates the overall material and finish of the house (Likert Scale)
        - OverallCond: Rates the overall condition of the house (Likert Scale)
    - Alphabetic:   
        - MSZoning: Identifies the general zoning classification of the sale.
        - Street: Type of road access to property
        - Alley: Type of alley access to property
        - LotShape: General shape of property
        - LandContour: Flatness of the property
        - Utilities: Type of utilities available
        - LotConfig: Lot configuration
        - LandSlope: Slope of property
        - Neighborhood: Physical locations within Ames city limits
        - Condition1: Proximity to various conditions
        - Condition2: Proximity to various conditions (if more than one is present)
        - BldgType: Type of dwelling
        - HouseStyle: Style of dwelling
        - RoofStyle: Type of roof
        - RoofMatl: Roof material
        - Exterior1st: Exterior covering on house
        - Exterior2nd: Exterior covering on house (if more than one material)
        - MasVnrType: Masonry veneer type
        - ExterQual: Evaluates the quality of the material on the exterior 
        - ExterCond: Evaluates the present condition of the material on the exterior
        - Foundation: Type of foundation
        - BsmtQual: Evaluates the height of the basement
        - BsmtCond: Evaluates the general condition of the basement
        - BsmtExposure: Refers to walkout or garden level walls
        - BsmtFinType1: Rating of basement finished area
        - BsmtFinType2: Rating of basement finished area (if multiple types)
        - Heating: Type of heating
        - HeatingQC: Heating quality and condition
        - CentralAir: Central air conditioning (Y/N - This could be changed to 0/1 bool)
        - Electrical: Electrical system
        - KitchenQual: Kitchen quality
        - Functional: Home functionality (Assume typical unless deductions are warranted)
        - FireplaceQu: Fireplace quality
        - GarageType: Garage location
        - GarageFinish: Interior finish of the garage
        - GarageQual: Garage quality
        - GarageCond: Garage condition
        - PavedDrive: Paved driveway
        - PoolQC: Pool quality
        - Fence: Fence quality
        - MiscFeature: Miscellaneous feature not covered in other categories
        - SaleType: Type of sale
        - SaleCondition: Condition of sale    
- Numerical Features:
    - LotFrontage: Linear feet of street connected to property
    - LotArea: Lot size in square feet
    - YearBuilt: Original construction date
    - YearRemodAdd: Remodel date (same as construction date if no remodeling or additions)
    - MasVnrArea: Masonry veneer area in square feet
    - BsmtFinSF1: Type 1 finished square feet
    - BsmtFinSF2: Type 2 finished square feet
    - BsmtUnfSF: Unfinished square feet of basement area
    - TotalBsmtSF: Total square feet of basement area
    - 1stFlrSF: First Floor square feet
    - 2ndFlrSF: Second floor square feet
    - LowQualFinSF: Low quality finished square feet (all floors)
    - GrLivArea: Above grade (ground) living area square feet
    - BsmtFullBath: Basement full bathrooms
    - BsmtHalfBath: Basement half bathrooms
    - FullBath: Full bathrooms above grade
    - HalfBath: Half baths above grade
    - Bedroom: Bedrooms above grade (does NOT include basement bedrooms)
    - Kitchen: Kitchens above grade
    - TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
    - Fireplaces: Number of fireplaces
    - GarageYrBlt: Year garage was built
    - GarageCars: Size of garage in car capacity
    - GarageArea: Size of garage in square feet
    - WoodDeckSF: Wood deck area in square feet
    - OpenPorchSF: Open porch area in square feet
    - EnclosedPorch: Enclosed porch area in square feet
    - 3SsnPorch: Three season porch area in square feet
    - ScreenPorch: Screen porch area in square feet
    - PoolArea: Pool area in square feet
    - MiscVal: Value of miscellaneous feature
    - MoSold: Month Sold (MM)
    - YrSold: Year Sold (YYYY)

I would like to examine all of the different types (specifically what numeric types there are).

In [7]:
training_data.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinSF1         int64
BsmtFinType2      object
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
Heating           object


In order to do some distribution analysis, I will filter out the non-numeric features 

In [8]:
numeric_training_data = training_data.select_dtypes(include=['int64','float64'])
numeric_dev_data = dev_data.select_dtypes(include=['int64','float64'])
print("Numeric Training Data Shape: ", numeric_training_data.shape)
print("Numeric Dev Data Shape: ", numeric_dev_data.shape)

Numeric Training Data Shape:  (1168, 38)
Numeric Dev Data Shape:  (292, 38)


It appears that the only (numeric) features with missing data are LotFrontage, Masonry veneer area in square feet (MasVnrArea), Garage Year Built (GarageYrBlt). Since lot frontage is the length of the property that is connected to the street, I will drop it. I believe that this value would hold similar information as the lot area. I'll also drop the Id (as it has no real numerical meaning), as well as the numeric categorical data.

In [9]:
numeric_training_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1168.000000,1168.000000,955.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1161.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1102.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000
mean,720.967466,57.551370,69.854450,10455.229452,6.128425,5.577911,1971.928938,1985.370719,102.824289,444.156678,49.229452,563.974315,1057.360445,1158.101884,342.011130,5.311644,1505.424658,0.428082,0.058219,1.566781,0.387842,2.856164,1.044521,6.477740,0.603596,1979.133394,1.772260,473.627568,94.059932,46.016267,21.696062,3.736301,14.987158,0.993151,48.410959,6.297089,2007.844178,180341.136130
std,425.354878,43.109105,23.450308,9826.869743,1.340406,1.097208,30.181188,20.360851,176.344349,433.039518,169.423106,440.111579,417.355147,378.425723,429.678949,44.491823,497.399012,0.515367,0.241463,0.546698,0.503039,0.791902,0.210449,1.563131,0.636955,24.374675,0.739057,209.740463,124.252934,65.808326,60.831986,30.944165,55.360939,24.154883,545.980770,2.704199,1.328496,75892.085404
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,372.000000,0.000000,0.000000,480.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,351.750000,20.000000,59.500000,7526.250000,5.000000,5.000000,1954.000000,1968.000000,0.000000,0.000000,0.000000,222.500000,794.000000,866.000000,0.000000,0.000000,1133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1962.000000,1.000000,338.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,130000.000000
50%,715.500000,50.000000,70.000000,9525.000000,6.000000,5.000000,1974.500000,1994.000000,0.000000,387.000000,0.000000,471.500000,991.000000,1080.000000,0.000000,0.000000,1457.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.500000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,165000.000000
75%,1090.500000,70.000000,80.000000,11515.500000,7.000000,6.000000,2001.000000,2004.000000,168.000000,719.250000,0.000000,795.250000,1302.000000,1382.250000,728.000000,0.000000,1767.250000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,65.250000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1459.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2009.000000,2010.000000,1600.000000,2260.000000,1474.000000,2336.000000,3206.000000,3228.000000,1872.000000,513.000000,4676.000000,2.000000,2.000000,3.000000,2.000000,6.000000,2.000000,12.000000,3.000000,2010.000000,4.000000,1390.000000,857.000000,547.000000,552.000000,508.000000,480.000000,648.000000,15500.000000,12.000000,2010.000000,755000.000000


In [10]:
numeric_dev_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,292.000000,292.000000,246.000000,292.000000,292.000000,292.000000,292.000000,292.000000,291.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,277.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000
mean,768.630137,54.280822,70.808943,10763.222603,5.982877,5.565068,1968.623288,1982.845890,107.120275,441.571918,35.828767,580.304795,1057.705479,1180.726027,366.917808,7.976027,1555.619863,0.414384,0.054795,1.558219,0.363014,2.907534,1.054795,6.678082,0.650685,1976.010830,1.746575,470.390411,94.982877,49.236301,22.986301,2.102740,15.356164,9.821918,23.801370,6.421233,2007.702055,183241.434932
std,404.749832,38.858231,27.322714,10590.397005,1.538637,1.175003,30.196639,21.665059,199.082350,539.402735,123.371083,449.346252,516.159126,417.843679,463.149438,62.524575,624.697933,0.533611,0.227969,0.568383,0.502641,0.905317,0.256351,1.847948,0.674447,25.799883,0.780445,229.705740,129.807535,68.071538,62.349749,21.613992,57.412026,75.437709,195.280048,2.703687,1.322653,92399.905647
min,5.000000,20.000000,21.000000,1596.000000,1.000000,2.000000,1890.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,1910.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,39300.000000
25%,443.750000,20.000000,58.250000,7716.000000,5.000000,5.000000,1949.000000,1962.000000,0.000000,0.000000,0.000000,232.250000,807.500000,907.750000,0.000000,0.000000,1125.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1958.000000,1.000000,291.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2006.000000,129000.000000
50%,780.500000,50.000000,69.000000,9174.500000,6.000000,5.000000,1969.000000,1992.000000,0.000000,347.000000,0.000000,499.500000,992.500000,1116.000000,0.000000,0.000000,1473.500000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1978.000000,2.000000,475.000000,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,155000.000000
75%,1112.750000,70.000000,80.000000,11839.500000,7.000000,6.000000,1998.000000,2003.000000,149.000000,690.000000,0.000000,870.000000,1284.000000,1423.000000,742.000000,0.000000,1804.500000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,8.000000,1.000000,2000.000000,2.000000,588.000000,168.000000,78.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,213122.500000
max,1460.000000,190.000000,313.000000,159000.000000,10.000000,9.000000,2010.000000,2010.000000,1378.000000,5644.000000,1057.000000,2046.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,1.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,736.000000,502.000000,330.000000,290.000000,440.000000,738.000000,2500.000000,12.000000,2010.000000,745000.000000


### Feature Engineering

In [11]:
numeric_training_data = numeric_training_data.drop(columns=['Id','MasVnrArea', 'LotFrontage', 'MSSubClass', 'OverallQual', 'OverallCond', 'GarageYrBlt'])
numeric_dev_data = numeric_dev_data.drop(columns=['Id','MasVnrArea', 'LotFrontage', 'MSSubClass', 'OverallQual', 'OverallCond', 'GarageYrBlt'])
print("Reduced Features: ", numeric_training_data.columns)
print("Training Data Shape: ", numeric_training_data.shape)
print("Dev Data Shape: ", numeric_dev_data.shape)

Reduced Features:  Index(['LotArea', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')
Training Data Shape:  (1168, 31)
Dev Data Shape:  (292, 31)


In [12]:
numeric_training_data.describe()

,LotArea,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000,1168.000000
mean,10455.229452,1971.928938,1985.370719,444.156678,49.229452,563.974315,1057.360445,1158.101884,342.011130,5.311644,1505.424658,0.428082,0.058219,1.566781,0.387842,2.856164,1.044521,6.477740,0.603596,1.772260,473.627568,94.059932,46.016267,21.696062,3.736301,14.987158,0.993151,48.410959,6.297089,2007.844178,180341.136130
std,9826.869743,30.181188,20.360851,433.039518,169.423106,440.111579,417.355147,378.425723,429.678949,44.491823,497.399012,0.515367,0.241463,0.546698,0.503039,0.791902,0.210449,1.563131,0.636955,0.739057,209.740463,124.252934,65.808326,60.831986,30.944165,55.360939,24.154883,545.980770,2.704199,1.328496,75892.085404
min,1300.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,372.000000,0.000000,0.000000,480.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,7526.250000,1954.000000,1968.000000,0.000000,0.000000,222.500000,794.000000,866.000000,0.000000,0.000000,1133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1.000000,338.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,130000.000000
50%,9525.000000,1974.500000,1994.000000,387.000000,0.000000,471.500000,991.000000,1080.000000,0.000000,0.000000,1457.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,165000.000000
75%,11515.500000,2001.000000,2004.000000,719.250000,0.000000,795.250000,1302.000000,1382.250000,728.000000,0.000000,1767.250000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2.000000,576.000000,168.000000,65.250000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,215245.000000,2009.000000,2010.000000,2260.000000,1474.000000,2336.000000,3206.000000,3228.000000,1872.000000,513.000000,4676.000000,2.000000,2.000000,3.000000,2.000000,6.000000,2.000000,12.000000,3.000000,4.000000,1390.000000,857.000000,547.000000,552.000000,508.000000,480.000000,648.000000,15500.000000,12.000000,2010.000000,755000.000000


I also want to remove any columns that might be linear combinations. (TODO)
- BsmtUnfSF: Unfinished square feet of basement area and TotalBsmtSF: Total square feet of basement area
- I'll also remove MiscVal because there's not much interpretability 
- Consider combining deck/porch 
- Consider combining square feet of floors 
- Consider combing features in general (number of bathrooms, kitchens, floors etc) 

In [13]:
# Split into data and labels
train_data = numeric_training_data.drop(columns=["SalePrice"])
train_labels = numeric_training_data["SalePrice"]
dev_data = numeric_dev_data.drop(columns=["SalePrice"])
dev_labels = numeric_dev_data["SalePrice"]
test_data = test_df

# Double check the shape
print("Training Data Shape: ", train_data.shape)
print("Training Data Shape: ", dev_data.shape)
print("Training Labels Shape: ", train_labels.shape)
print("Training Labels Shape: ", dev_labels.shape)
print("Test Data Shape: ", test_data.shape)

Training Data Shape:  (1168, 30)
Training Data Shape:  (292, 30)
Training Labels Shape:  (1168,)
Training Labels Shape:  (292,)
Test Data Shape:  (1459, 80)


In [14]:
# Scale the Data
scaler = StandardScaler()
norm = scaler.fit(train_data)
train_data = norm.transform(train_data)
print(train_data.shape)
# Normalize features using the standard scaler [dev data]
dev_data = norm.transform(dev_data)
print(dev_data.shape)

(1168, 30)
(292, 30)


### Models

#### Random Forest

In [15]:
# Try a random forest 
def RandomForest(num_trees):
    model = RandomForestClassifier(num_trees,max_depth=8)
    model.fit(train_data, train_labels)
    predictions = model.predict(dev_data)
    score = model.score(dev_data, dev_labels)
    print("Random Forest Performance for {0} trees: {1}".format(num_trees,score))
    # Plot_confusion_matrix
#     plot_confusion_matrix(model, dev_data, dev_labels, values_format = "d")
#     plt.title("{} Tree Random Forest Confusion Matrix:".format(num_trees))
#     plt.plot()
    
num_trees_list = [1,3,5,10,100]
for num_trees in num_trees_list:
    RandomForest(num_trees)

Random Forest Performance for 1 trees: 0.0
Random Forest Performance for 3 trees: 0.010273972602739725
Random Forest Performance for 5 trees: 0.0136986301369863
Random Forest Performance for 10 trees: 0.003424657534246575
Random Forest Performance for 100 trees: 0.003424657534246575


#### Linear Regression

In [36]:
def LinearRegressionModel():
    model = LinearRegression()
    model.fit(train_data, train_labels)
    dev_predicted = model.predict(dev_data)
    score = model.score(dev_data, dev_labels)
    print("Mean Accuracy: ", score)
    
LinearRegressionModel()

Mean Accuracy:  0.6214273597507431


#### Logistic Regression

In [33]:
def LogRegressionModel():
#     model = LogisticRegression(penalty='l2', C=0.5, solver="liblinear", multi_class="auto")
    model = LogisticRegression()
    model.fit(train_data, train_labels)
    dev_predicted = model.predict(dev_data)
    score = model.score(dev_data, dev_labels)
    f1_score = metrics.f1_score(dev_labels, dev_predicted, average='weighted')
    print("F1 Score: {0} Mean Accuracy: {1}".format(f1_score, score))

LogRegressionModel()

TypeError: LogisticRegression() got an unexpected keyword argument 'penalty'

#### Multi-Layer Perceptron

In [ ]:
def MLP():
    # Default activation is 'relu', random state lets us get the same result every time (so we can tune other parameters)
    # max_iter is 200 by default, but more helps. alpha is the regularization parameter. solver is 'adam' by default
    model = MLPClassifier(alpha=1e-3, hidden_layer_sizes=(100,), random_state=0, max_iter=300) 
    model.fit(train_data, train_labels) 
    score = model.score(dev_data, dev_labels)
    print("MLP accuracy = ",score)
MLP()

#### K-Nearest Neighbors

In [17]:
# Try K Nearest Neighbors - before any data cleaning 
def KNN(kn):
    model = KNeighborsClassifier(n_neighbors = kn)
    model.fit(train_data, train_labels)
    predictions = model.predict(dev_data)
    score = model.score(dev_data, dev_labels)
    print("KNN {0} neighbors : accuracy = {1}".format(kn,score))
    
# The alpha isn't actually making a difference 
neigh_list = [1,2,4, 7, 10]
for neigh in neigh_list:
    KNN(neigh)

KNN 1 neighbors : accuracy = 0.003424657534246575
KNN 2 neighbors : accuracy = 0.003424657534246575
KNN 4 neighbors : accuracy = 0.003424657534246575
KNN 7 neighbors : accuracy = 0.003424657534246575
KNN 10 neighbors : accuracy = 0.00684931506849315
